# Luokittelu

JAMK examples #10: 10_ml_revenue_classification

Sisältää mm.

- luokittelumallin suorituskyvyn perusraportti

Ladataan käytettävät paketit.

In [48]:

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

Luetaan käytettävä aineisto:

In [49]:
csvUrl = "https://raw.githubusercontent.com/RaimoHaikari/paikkatieto/master/misc/sales_data.csv"
df = pd.read_csv(csvUrl)

df.shape

(1000, 6)

Seuraavaksi:

- tiputetaan NA-arvoja sisältävät tietueet
- muutetaan date-sarake aikaleimaksi
- lasketaan päivittäinen liikevaihto
- muunnetaan aineisto aikasarjaksi

In [50]:
# Handle missing values
# df = df.dropna()
df = df.dropna().reset_index()

# Convert the 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

# Calculate the daily revenue
df['revenue'] = df['sales'] * df['price']

df.set_index('date', inplace=True)

Luokitellaan aineisto liikevaihdon mukaan kolmeen luokkaan:

- **Low** myynti on alle 1000 
- **Medium** myynti on välillä 1000 ... 5000
- **High** myynti on yli 5000

In [51]:
# Define revenue classes based on limits
low_limit = 1000
high_limit = 5000
df['revenue_class'] = pd.cut(
    df['revenue'], 
    bins=[-float('inf'), low_limit, high_limit, float('inf')],
    labels=['Low', 'Medium', 'High']
)

Erotetaan syöte- ja vastemuuttujan omiin muuttujiinsa.

In [52]:
# Prepare the data
X = df[['price', 'category']]  # Features
y = df['revenue_class']  # Target variable

Dummy-koodataa luokkamuuttujat.

In [53]:
# Perform one-hot encoding on the 'category' column
X = pd.get_dummies(X)

Jaetaan aineisto opetus- ja testijoukkoihin.

In [54]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Alustetaan luokittelua varten RandomForrest -luokittelija, jonka jälkeen sovitetaan malli.

In [55]:
# Initialize the random forest classifier model
model = RandomForestClassifier()

model.fit(X_train, y_train)

RandomForestClassifier()

RandomForrest objektin feature_importances_ -attibuutin takaa saadaan esille käytettyjen muuttujien merkitys mallin kannalta. 

Napataan arvot ja muuttujien nimet omaan kehikkoonsa.

Lajitellaan taulukko suurimmasta pienimpään.

In [56]:
# Get feature importances
feature_importances = model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Sort the DataFrame by importance values in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=True)
print(round(feature_importance_df, 3))

              Feature  Importance
1      category_Books       0.005
4   category_Monitors       0.005
5     category_Phones       0.007
3  category_Computers       0.007
2   category_Clothing       0.009
0               price       0.967


Testataan mallia testijoukolla.

In [57]:
# Make predictions on the test set
y_pred = model.predict(X_test)

sklearn sisältää [classification_report-funktion](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html), joka tulostaa luokittelusta perusrapsan:

- **precision** kertoo onnistuneiden ennusteiden osuuden (ENNUSTE = TODELLINEN)
- **recall** kertoo kuinka hyvin todellisien arvojen näkökulmasta ennusteet osuivat (TODELLINEN = ENNUSTE)
- **f1-score** on precision ja recall arvojen yhdistelmä. (Näyttäisi olevan puolessa välissä)
- **support** luokkien todellisten ilmentymien lkm
- **accuracy** kuinka suuri osuus ennusteista osui oikeaan, kun huomioidaan kaikki luokat

- **macro avg** kaikkien ennusteluokkien onnistumisprosenttien keskiarvo, niin ennusteen, todellisten kuin näiden yhteenvedon näkökulmasta.
- **weighted avg** painottaa ennusteiden arvoja luokkafrekvenssien mukaisesti, [mitä ei yleensä haluta, koska harvinaisemmat luokat yleensä vaikeampia tunnistaa](https://datascience.stackexchange.com/questions/65839/macro-average-and-weighted-average-meaning-in-classification-report).


In [59]:
# Evaluate the model
classification_report = classification_report(y_test, y_pred)
print('Classification Report:\n', classification_report)

Classification Report:
               precision    recall  f1-score   support

        High       0.44      0.44      0.44        18
         Low       0.61      0.52      0.56        33
      Medium       0.65      0.70      0.68        61

    accuracy                           0.61       112
   macro avg       0.57      0.55      0.56       112
weighted avg       0.61      0.61      0.60       112



In [68]:
foo = pd.DataFrame({'pred': y_pred,'is': y_test })

# table = pd.pivot_table(df, values='D', index=['A', 'B'], columns=['C'], aggfunc="sum")
# pd.pivot_table(foo, values='pred', index=['pred'], columns=['is'], aggfunc="count")
# foo.groupby('pred').count()
foo.head()

,pred,is
date,,
2023-06-23,Medium,Low
2023-02-09,Low,Low
2023-05-17,Medium,Medium
2023-06-25,Low,Low
2023-06-03,High,Medium
